## Este codigo será corregido para que funcione tanto en las anotaciones de cristalografía (en su mayoría sacadas con DSSP) como en las de STRIDE.
## También se hará la fusión de residuos y hebras.
## Los residuos continuos y separados por un solo número, deberán pertenecer a la misma hebra.

In [2]:
import pandas as pd
#from sklearn import linear_model
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from sklearn.linear_model import LinearRegression
from skspatial.objects import Points, Line, Plane, Vector, Point
from skspatial.plotting import plot_3d
import copy
import scipy
import statistics 
from pathlib import Path
import sys
from skimage.measure import EllipseModel
from ast import literal_eval
from biopandas.pdb import PandasPdb
import sklearn

In [3]:
#archivo_prefijo = archivo.split('_')[0]+'.SHEETaln.txt'
#aln= pd.read_table(path/archivo_prefijo, header=None)#, index_col=0)
#CA.loc[CA.index.intersection((aln.index).values)] # imprime los indices que se intersec

In [4]:
print(pd.__version__)

1.2.4


# FUNCIONES

### 0. Formación de hebras por residuos continuos (un residuo de separación máximo) y fusión de datos cristalografía (DSSP) y STRIDE.

In [89]:
def numeracion_hebras(df):
    random_iterator = iter(df.index)
    b=next(random_iterator)
    c=1
    while c < len(df):
        c=c+1
        a=b
        b=next(random_iterator)
        if df.at[b,'hebra'] == df.at[a,'hebra']:
            pass
        elif abs(df['num_res'][b] - df['num_res'][a])<=2: 
            df.at[b,'hebra']=df.at[a,'hebra']

### 1. Lee el archivo con información de las hebras beta del tim barrel, crea un dataframe sólo con los CA y cuenta el número de hebras que tiene el tim barrel.

In [88]:
def datos(archivo):
    txt = pd.read_table(path/archivo, names=colnames, header=None)#, index_col='num_res') #Uso como indice num_res
    print(archivo)
    
    CA=txt[txt['atomo']=='CA']
    CB=txt[txt['atomo']=='CB']
    N=txt[txt['atomo']=='N']
    O=txt[txt['atomo']=='O']
    
    numeracion_hebras(CA)
    numeracion_hebras(CB)
    numeracion_hebras(N)
    numeracion_hebras(O)
    
    n_hebras=CA['hebra'].nunique() #numero de hebras
    return n_hebras, CA, txt, CB, N, O
#Guardar el número de hebras inicial que tienen para posterior comparación con anotaciones de STRIDE y el futuro combinado
#Posteriormente haremos un merge para unir los df de CA y CB

### 2. Crea los ejes que representan mejor a las hebras beta del tim barrel usando svd.

In [86]:
def fitlines(CA):
    dic_fit={}
    for i in CA["hebra"].unique(): #lista con todas las hebras diferentes
        coord=CA.loc[CA['hebra']==i, ['x','y','z']]
        coord_mat= coord.to_numpy()        
        #line_fit= Line.best_fit(coord_mat) #fit a line to points
        coord_mat = Points(coord_mat)
        points_centered, centroid =coord_mat.mean_center(return_centroid=True)
        u,s,vh = np.linalg.svd(points_centered, float)     
        dic_fit[i]=Line(point=centroid, direction= vh[0, :])
    return dic_fit

### 3. Encuentra el ángulo entre dos vectores.

In [85]:
def angle_between(v1, v2):
    dot_pr = v1.dot(v2)
    norms = np.linalg.norm(v1) * np.linalg.norm(v2)
    return np.rad2deg(np.arccos(dot_pr / norms))

### 4. Calcula la distancia y el ángulo entre los ejes de las hebras. Calcula la distancia entre los CA de las hebras más cercanas y con un ángulo dentro del umbral. Si la distancia entre un par de CA de dos hebras diferentes es menor a 6A, entonces define que esas hebras son adyacentes.

In [298]:
def distancia_interhebra_angulo(dic_fit, CA, N, O, primera):
    dic_dis={}
    for i in CA["hebra"].unique():
        #Obtencion de coordenadas de la hebra i
        CAcoordi=(CA.loc[CA['hebra']==i, ['x','y','z']]).to_numpy()
        Ncoordi=(N.loc[N['hebra']==i, ['x','y','z']]).to_numpy()  
        Ocoordi=(O.loc[O['hebra']==i, ['x','y','z']]).to_numpy()  
        dic_dis[i]={}
        for j in CA["hebra"].unique():
            #Obtencion de coordenadas de la hebra j
            CAcoordj=(CA.loc[CA['hebra']==j, ['x','y','z']]).to_numpy()
            Ncoordj=(N.loc[N['hebra']==j, ['x','y','z']]).to_numpy()  
            Ocoordj=(O.loc[O['hebra']==j, ['x','y','z']]).to_numpy()
            
            if j!=i and CAcoordi.size!=0 and Ncoordi.size!=0 and Ocoordi.size!=0 and CAcoordj.size!=0 and Ncoordj.size!=0 and Ocoordj.size!=0:

                #Calcular distancia euclidiana    
                #Filas coordi, columnas coordj
                CA_dis = sklearn.metrics.pairwise.euclidean_distances(CAcoordi,CAcoordj)
                CA_dis_bol = CA_dis < 6
                CA_dis_true = CA_dis_bol.sum()

                N_O_dis = sklearn.metrics.pairwise.euclidean_distances(Ncoordi,Ocoordj)
                N_O_dis_bol = N_O_dis <  3.5     
                N_O_dis_true = N_O_dis_bol.sum()

                O_N_dis = sklearn.metrics.pairwise.euclidean_distances(Ocoordi,Ncoordj)
                O_N_dis_bol = O_N_dis < 3.5
                O_N_dis_true = O_N_dis_bol.sum()

                d= dic_fit[i].distance_line(dic_fit[j])
                a= angle_between(dic_fit[i].vector, dic_fit[j].vector)

                condicion= ((CA_dis_true>1 or (N_O_dis_true or O_N_dis_true)) and (a<60 or 120<a<180)) if primera else (CA_dis_true>1 or (N_O_dis_true or O_N_dis_true))
                if condicion: # True CA<6A y angulo CA<60  # False Que haya al menos una pareja de CA<6A o un par de N-O u O-N.
                    if CA_dis_true:
                        dic_dis[i][j]=(CA_dis[CA_dis_bol]).mean(), a, d # Para contemplar si hay valores CA-CA<6
                    else:
                        dic_dis[i][j]=np.nan, a, d 

            #else: 
             #   print(i,'\t', j,'\t', d, a)
                
    return dic_dis

### 5. A partir de los pares de hebras adyacentes, crea grupos de hebras que estén conectadas consecutivamente.

In [10]:
def ciclo(T,I,T_FIJA, lista_keys, dic_dis): #hace grupos con las hebras conectadas
    for t_prima in dic_dis[T].keys():
        if t_prima not in lista_keys[I][T_FIJA]:
            lista_keys[I][T_FIJA].append(t_prima)
            #print('|')
            #print('padre:',T,' - hijo:',t_prima)
            ciclo(t_prima, I, T_FIJA, lista_keys, dic_dis)

def grupos_hebras_conectadas(dic_dis):
    lista_keys={} #hace grupos con hebras conectadas, trabaja recursivamente con la funcion ciclo
    for i in dic_dis.keys():
        lista_keys[i]={}
        #print('_____________\nraiz:',i)
        #print('|')
        for t_fija in dic_dis[i].keys():
            lista_keys[i][t_fija]=[i, t_fija]
            #print('lista nueva: ','padre y raiz:',i,' - hijo y subraiz:',t_fija)
            ciclo(t_fija, i, t_fija, lista_keys, dic_dis)
    return lista_keys


### 6. Cuenta el número de hebras que tiene cada grupo y dime cuál es el grupo con más hebras. Premisa: el grupo con más hebras es el barril del tim barrel.

In [11]:
def grupo_mas_grande(grupos):  #Encuentra el grupo con más hebras. En teoría sería el barril.
    max_num_hebras=0         
    for i in grupos.keys():
        for j in grupos[i].keys():
            if len(grupos[i][j]) > max_num_hebras:
                max_num_hebras=len(grupos[i][j])
                max_key_i=i
                max_key_j=j
    hebras_barril = grupos[max_key_i][max_key_j]
    return max_num_hebras, max_key_i, max_key_j, hebras_barril

### 7. Dime si la primer hebra del barril se conecta con la última hebra. Si es un circuito cerrado de hebras (la primera se conecta con la última), entonces comprobamos que es un barril. De lo contrario, puede tratarse de un barril incompleto o puede que no se haya escogido la hoja beta correcta.

In [12]:
def barril_circuito(hebras_barril, dic_dis): #La primer hebra del barril tiene
    if hebras_barril[-1] in dic_dis[hebras_barril[0]]: #que conectarse con la ultima
        circuito=1      #para ser circuito cerrado . Cerrado =1, abierto =0
    else:
        circuito=0
    return circuito

### 8. Crea un dataframe con las coordenadas de CA del barril. Actualiza los números de las hebras

In [13]:
def coordenadas_barril_actualizacion(hebras_barril, CA, N, O): #creando un df que contenga las coordenadas
    CA_barril=CA[CA['hebra'].isin(hebras_barril)] # de las hebras del barril   
    N_barril=N[N['hebra'].isin(hebras_barril)]
    O_barril=O[O['hebra'].isin(hebras_barril)]
    
    CA_barril = CA_barril.assign(hebra_old=pd.Series(CA_barril['hebra'])) #Guardando registro de la vieja numeración  
    N_barril = N_barril.assign(hebra_old=pd.Series(N_barril['hebra']))
    O_barril = O_barril.assign(hebra_old=pd.Series(O_barril['hebra']))
    
    rango=list(range(1,len(hebras_barril)+1)) #Creando lista con los nuevos números de las hebras del 1 al 8
    
    dict_rango_CA = dict(zip(CA_barril["hebra"].unique(),rango))  #Enumeramos a las hebras del barril de acuerdo al orden de la secuencia de aminoácidos, convertimos dos listas en un diccionario
    dict_rango_N = dict(zip(N_barril["hebra"].unique(),rango))
    dict_rango_O = dict(zip(O_barril["hebra"].unique(),rango))
    
    CA_barril['hebra']=CA_barril['hebra'].map(dict_rango_CA) #Cambiamos el nombre de las hebras
    N_barril['hebra']=N_barril['hebra'].map(dict_rango_N)
    O_barril['hebra']=O_barril['hebra'].map(dict_rango_O)    
        
    return CA_barril, N_barril, O_barril, rango

### 9. Extrae los vectores de direccion y centroides de los ejes de las hebras, y con sus promedios calcula el eje central.

In [14]:
def centros_direcciones_barril(dic_fit_new, rango):
    #Extraer las direcciones y los centroides de las hebras del barril; calcular sus medias.
    direcciones=np.array([(dic_fit_new[i].direction) for i in rango])
    direcciones_heb_barril=pd.DataFrame(direcciones,columns=['x', 'y', 'z'], index=rango)
    
    centros=np.array([(dic_fit_new[i].point) for i in rango])
    centros_heb_barril=pd.DataFrame(centros,columns=['x', 'y', 'z'], index=rango)
    
    eje_vector=direcciones.mean(axis=0)
    eje_centro=centros.mean(axis=0)    
    eje_central = Line(point=eje_centro, direction=eje_vector)    
    return direcciones_heb_barril, centros_heb_barril, eje_central

### 10. Calcula el RMSD enter los CA de las hebras del barril y sus líneas ajustadas. Calcula la distancia promedio entre CA-CA para cada hebra. Calcula la distancia mínima promedio entre los CA de hebras adyacentes. Inicializa el dataframe que incluye los datos de todas las hebras.

In [15]:
def a_b_rmsd_hebras(rango, dic_dis_new, CA_barril, df_1, dic_fit_new):        
    for i in rango:
        coordi=(CA_barril.loc[CA_barril['hebra']==i, ['x','y','z']]).to_numpy()
        error = np.array([dic_fit_new[i].distance_point(ri) for ri in coordi]) ### RMSD
        df_1.loc[(archivo.split('_')[0],i),'rmsd']= np.sqrt(np.mean(error**2))
        
        ci, ai_suma = 0, 0 #Saca la distancia promedio entre CA de las mismas hebras
        if len(coordi)>1:
            for ri in range(len(coordi)-1):
                distancia = Point(coordi[ri]).distance_point(Point(coordi[ri+1]))
                ai_suma = ai_suma + distancia
                ci=ci+1
            ai_prom = ai_suma/ci
            df_1.loc[(archivo.split('_')[0],i),'a']= ai_prom
                    
        ci, bi_suma = 0, 0 #Saca la distancia minima promedio entre hebras
        for j in rango:
            if j in dic_dis_new[i]:
                bi_suma = bi_suma + dic_dis_new[i][j][0]
                ci=ci+1
        bi_prom = bi_suma/ci       
        df_1.loc[(archivo.split('_')[0],i), 'b'] = bi_prom        
    return df_1

### 11. Calcula el número de shear usando proyección de punto a linea 

In [16]:
def shear_proyeccion(dic_fit, hebras_barril, a):
    punto_inicial =dic_fit[hebras_barril[-1]].point
    punto =dic_fit[hebras_barril[-1]].point
    for i in hebras_barril:
        punto = dic_fit[i].project_point(punto)
    shear= punto_inicial.distance_point(punto) / a
    return shear

### 12. Calcula el ángulo de twist entre las hebras adyacentes del barril

In [17]:
def angulo_twist(df_angulo_twist, rango, dic_dis_new):
    for i in rango:
        for j in rango:
            if j in dic_dis_new[i]:
                tw = 180-dic_dis_new[i][j][1]  if dic_dis_new[i][j][1] > 90 else dic_dis_new[i][j][1]
                df_angulo_twist=df_angulo_twist.append({'barril':archivo.split('_')[0], 'hebra1':i, 'hebra2':j, 'angulo':tw}, ignore_index=True)
    return df_angulo_twist

### Calcula el ángulo entre las hebras adyacentes del barril, considerando que la conexión entre las hebras sigue el orden (ascendente) de la secuencia de aminoácidos.
Tengo esta premisa ya que todos los barriles que he visto respetan ese orden. Sólo aplico esto a barriles con 8 hebras

In [83]:
def angulo_conexion(df_angulo_conexion,CA_barril,N_barril,O_barril,dic_fit_new):
    dic_ac={}
    for i in CA_barril["hebra"].unique():
        CAi=(CA_barril.loc[CA_barril['hebra']==i, ['x','y','z']]).to_numpy()
        Ni=(N_barril.loc[N_barril['hebra']==i, ['x','y','z']]).to_numpy()  
        Oi=(O_barril.loc[O_barril['hebra']==i, ['x','y','z']]).to_numpy()    
        dic_ac[i]={}
        
        j = 1 if i==8 else i+1

        CAj=(CA_barril.loc[CA_barril['hebra']==j, ['x','y','z']]).to_numpy()
        Nj=(N_barril.loc[N_barril['hebra']==j, ['x','y','z']]).to_numpy()  
        Oj=(O_barril.loc[O_barril['hebra']==j, ['x','y','z']]).to_numpy()

        CA_dis_true = (sklearn.metrics.pairwise.euclidean_distances(CAi,CAj) < 6).sum()
        N_O_dis_true = (sklearn.metrics.pairwise.euclidean_distances(Ni,Oj) < 3.5).sum()
        O_N_dis_true = (sklearn.metrics.pairwise.euclidean_distances(Oi,Nj) < 3.5).sum()

        angulo= angle_between(dic_fit_new[i].vector, dic_fit_new[j].vector)
        angulo= 180-angulo if angulo > 90 else angulo

        dic_ac[i][j]= angulo, CA_dis_true, N_O_dis_true, O_N_dis_true

    df_angulo_conexion = df_angulo_conexion.append({'barril':archivo.split('_')[0],'tecnica':(archivo.split('SHEET')[1]).split('.')[0],
                                            'an_1_2':dic_ac[1][2][0],'an_2_3':dic_ac[2][3][0],'an_3_4':dic_ac[3][4][0],'an_4_5':dic_ac[4][5][0],'an_5_6':dic_ac[5][6][0],'an_6_7':dic_ac[6][7][0],'an_7_8':dic_ac[7][8][0],'an_8_1':dic_ac[8][1][0],
                                            'CA_1_2':dic_ac[1][2][1],'CA_2_3':dic_ac[2][3][1],'CA_3_4':dic_ac[3][4][1],'CA_4_5':dic_ac[4][5][1],'CA_5_6':dic_ac[5][6][1],'CA_6_7':dic_ac[6][7][1],'CA_7_8':dic_ac[7][8][1],'CA_8_1':dic_ac[8][1][1],
                                            'NO_1_2':dic_ac[1][2][2],'NO_2_3':dic_ac[2][3][2],'NO_3_4':dic_ac[3][4][2],'NO_4_5':dic_ac[4][5][2],'NO_5_6':dic_ac[5][6][2],'NO_6_7':dic_ac[6][7][2],'NO_7_8':dic_ac[7][8][2],'NO_8_1':dic_ac[8][1][2],
                                            'ON_1_2':dic_ac[1][2][3],'ON_2_3':dic_ac[2][3][3],'ON_3_4':dic_ac[3][4][3],'ON_4_5':dic_ac[4][5][3],'ON_5_6':dic_ac[5][6][3],'ON_6_7':dic_ac[6][7][3],'ON_7_8':dic_ac[7][8][3],'ON_8_1':dic_ac[8][1][3]}, 
                                           ignore_index=True)
    return df_angulo_conexion


### 5. Calcular las distancias entre los CA y el centroide del barril

In [19]:
#-------------------------------------------------------------------------------
# RODRIGUES ROTATION
# - Rotate given points based on a starting and ending vector
# - Axis k and angle of rotation theta given by vectors n0,n1
#   P_rot = P*cos(theta) + (k x P)*sin(theta) + k*<k,P>*(1-cos(theta))
#-------------------------------------------------------------------------------
def rodrigues_rot(P, n0, n1):
    
    # If P is only 1d array (coords of single point), fix it to be matrix
    if P.ndim == 1:
        P = P[np.newaxis,:]
    
    # Get vector of rotation k and angle theta
    n0 = n0/np.linalg.norm(n0)
    n1 = n1/np.linalg.norm(n1)
    k = np.cross(n0,n1)
    k = k/np.linalg.norm(k)
    theta = np.arccos(np.dot(n0,n1))
    
    # Compute rotated points
    P_rot = np.zeros((len(P),3))
    for i in range(len(P)):
        P_rot[i] = P[i]*np.cos(theta) + np.cross(k,P[i])*np.sin(theta) + k*np.dot(k,P[i])*(1-np.cos(theta))

    return P_rot

def fit_an_ellipse(P):
    P_mean = P.mean(axis=0)
    P_centered = P - P_mean
    
    # Fitting plane by SVD for the mean-centered data
    U,s,V = np.linalg.svd(P_centered, full_matrices=False)
    
    # Normal vector of fitting plane is given by 3rd column in V
    # Note linalg.svd returns V^T, so we need to select 3rd row from V^T
    # normal on 3d plane
    normal = V[2,:]
    
    # Project points to coords X-Y in 2D plane
    P_xy = rodrigues_rot(P_centered, normal, [0,0,1])
    
    # Use skimage EllipseModel to fit an ellipse to set of 2d points
    ell = EllipseModel()
    ell.estimate(P_xy[:, :2])
    
    # Generate n 2D points on the fitted elippse
    n = 100
    xy = ell.predict_xy(np.linspace(0, 2 * np.pi, n))
    
    # Convert the 2D generated points to the 3D space
    points = []
    for i in range(len(xy)):
        points.append([xy[i, 0], xy[i, 1], 0])
    points = np.array(points)
    ellipse_points_3d = rodrigues_rot(points, [0,0,1], normal) + P_mean
    
    return xy, ell, P_xy[:, :2]

In [20]:
def ajusta_elipse(CA_barril, eje_central):
    #Calcular los puntos (CA) más cercanos al centroide del eje central y probar elipse con ellos
    CA_barril['distancia'] = CA_barril[['x','y','z']].apply(lambda x: eje_central.point.distance_point(x) , axis=1)
    CA_waist=pd.DataFrame(columns=CA_barril.columns)
    for i in rango:
        CA_hebra= CA_barril.loc[CA_barril['hebra']==i]
        CA_waist= CA_waist.append(CA_hebra[CA_hebra['distancia'] == CA_hebra['distancia'].min()]) #Infomracion sobre los CA mas cercanos al centroide
    CA_waist.drop(['atomo','cadena','orient', 'sheet'], axis=1, inplace=True)

    P = np.array(CA_waist[['x','y','z']])
    ellipse_points, ell, P_xy = fit_an_ellipse(P)
    a=max(ell.params[2],ell.params[3])
    b=min(ell.params[2],ell.params[3])
    c=np.sqrt(a**2-b**2)
    e=c/a
    area=a*b*np.pi
    perim= math.pi*(3.0*(a+b)-math.sqrt((3.0*a+b)*(a+3.0*b)))
    rmsd_ajuste = np.sqrt(np.mean(ell.residuals(P_xy)**2))

    return CA_waist, e, area, rmsd_ajuste, perim

# Abrir y leer el/los archivos de entrada

In [299]:
path= Path('')
#path= Path('')
#path= Path('')
#path= Path('')

#with open(path/'lista.txt', 'r') as f:
#    lista_arc=[_.rstrip('\n') for _ in f]
    
path= Path('')
lista_arc=['2dh2A01_SHEETalncris.txt']

# Inicialización de los DataFrames

In [300]:
#1uuqA00 fue removido de sc
# quitados 2og9A02_SHEETsc.txt, 1tr9A00_SHEETsc.txt, 2ovlA02_SHEETsc.txt, 2qneA01_SHEETsc.txt, 2qw5A00_SHEETsc.txt, 3abzA01_SHEETsc.txt, 3dx5A00_SHEETsc.txt,
# quitados 3msyA02_SHEETsc.txt, 3n4eA02_SHEETsc.txt, 3tevA00_SHEETsc.txt, 4h41A00_SHEETsc.txt, 5jbdA02_SHEETsc.txt

colnames = ['hebra', 'atomo', 'res', 'cadena', 'num_res','x', 'y', 'z', 'orient', 'sheet'] # agregar nuevos encabezados

df_2=pd.DataFrame(columns = ['barril','tecnica','circuito','nraw','n','e','area','rmsd_ajuste','perim','alfa_prom','alfa_ds','rmsd','a_prom','a_ds','b_prom','b_ds',
                             'shear_McL','shear_proyec','R_McL','R_Murz','area_c','perim_c','twist_McL','angtw','angtw_ds']) #esto se tiene que definir al inicio de todas las corridas con todos los barriles
df_1_todo=pd.DataFrame()

df_angulo_twist=pd.DataFrame(columns=['barril','hebra1', 'hebra2', 'angulo'])

df_angulo_conexion = pd.DataFrame(columns=['barril','tecnica',
                                             'an_1_2','an_2_3','an_3_4','an_4_5','an_5_6','an_6_7','an_7_8','an_8_1',
                                             'CA_1_2','CA_2_3','CA_3_4','CA_4_5','CA_5_6','CA_6_7','CA_7_8','CA_8_1',
                                             'NO_1_2','NO_2_3','NO_3_4','NO_4_5','NO_5_6','NO_6_7','NO_7_8','NO_8_1',
                                             'ON_1_2','ON_2_3','ON_3_4','ON_4_5','ON_5_6','ON_6_7','ON_7_8','ON_8_1'])

# Procesamiento de datos, aplicación de funciones y llenado de dataframes

In [301]:
for archivo in lista_arc:
    
    #APLICACIÓN DE FUNCIONES PARA DEFINIR EL BARRIL
    n_hebras, CA, txt, CB, N, O = datos(archivo)
    dic_fit = fitlines(CA)
    dic_dis = distancia_interhebra_angulo(dic_fit, CA, N, O, True) # El primero con los filtros más duros para descartar hebras fuera del barril
    grupos = grupos_hebras_conectadas(dic_dis)
    tamano, mi, mj, hebras_barril = grupo_mas_grande(grupos)
    
    CA_barril, N_barril, O_barril, rango = coordenadas_barril_actualizacion(hebras_barril, CA, N, O) 
    dic_fit_new = fitlines(CA_barril)
    dic_dis_new = distancia_interhebra_angulo(dic_fit_new, CA_barril, N_barril, O_barril, False) # El segundo con un filtro más flexible para ver si el barril es cerrado
    
    grupos_new = grupos_hebras_conectadas(dic_dis_new)
    tamano_new, mi_new, mj_new, hebras_barril_new = grupo_mas_grande(grupos_new)
    
    circuito = barril_circuito(hebras_barril_new, dic_dis_new)
    
    direcciones_heb_barril, centros_heb_barril, eje_central = centros_direcciones_barril(dic_fit_new, rango)
    
    ### Solo calcula elipse si el barril tiene 5 o más hebras
    if tamano_new>4:
        CA_waist, e, area, rmsd_ajuste, perim = ajusta_elipse(CA_barril, eje_central)
    else: 
        CA_waist, e, area, rmsd_ajuste, perim = np.nan,np.nan,np.nan,np.nan,np.nan
    
    
    ### Construcción del dataframe con información de cada hebra de los barriles (df_1)
    index = pd.MultiIndex.from_product([[archivo.split('_')[0]], rango], names=['barril', 'hebra'])
    df_1=pd.DataFrame(index=index,columns = ['rmsd','a', 'b', 'alfa','len'])
    df_1=a_b_rmsd_hebras(rango, dic_dis_new, CA_barril, df_1, dic_fit_new)    
    df_1['alfa'] = [angle_between(direcciones_heb_barril.loc[i], eje_central.direction) for i in rango]
    df_1['alfa'] = [180-i if i>90 else i for i in df_1['alfa']]
    df_1['len'] = list(map(lambda x: len(CA_barril[CA_barril['hebra']==x]), rango))

    ### Construcción del dataframe con los ángulos twist entre hebras adyacentes del barril (df_angulo_twist)
    df_angulo_twist = angulo_twist(df_angulo_twist, rango, dic_dis_new) 
    df_tw= df_angulo_twist[df_angulo_twist['barril']==archivo.split('_')[0]]
    
    ### Construccion del dataframe con los angulos y conexiones ###
    if tamano_new == 8:
        df_angulo_conexion = angulo_conexion(df_angulo_conexion,CA_barril,N_barril,O_barril,dic_fit_new)

    ### Construccioón del dataframe df_2 con información de los barriles    
    alfa = df_1.loc[(archivo.split('_')[0]),'alfa'].mean(axis=0)
    a = df_1.loc[(archivo.split('_')[0]),'a'].mean(axis=0)
    b = df_1.loc[(archivo.split('_')[0]),'b'].mean(axis=0)
    shear_McL = tamano_new * b * math.tan(math.radians(alfa)) / a
    shear_proyec = shear_proyeccion(dic_fit_new, hebras_barril_new, a)
    R_McL = b / (2 * math.sin(math.pi / tamano_new) * math.cos(math.radians(alfa)))
    
    df_2=df_2.append({'barril':archivo.split('_')[0],
                      'tecnica':(archivo.split('SHEET')[1]).split('.')[0],
                      'circuito':circuito,
                      'nraw':n_hebras,
                      'n':tamano_new,
                      'e':e,
                      'area':area,
                      'rmsd_ajuste':rmsd_ajuste,
                      'perim':perim,
                      'alfa_prom':alfa,
                      'alfa_ds':df_1.loc[(archivo.split('_')[0]),'alfa'].std(axis=0),
                      'rmsd':df_1.loc[(archivo.split('_')[0]),'rmsd'].mean(axis=0),
                      'a_prom':a,
                      'a_ds':df_1.loc[(archivo.split('_')[0]),'a'].std(axis=0),
                      'b_prom':b,
                      'b_ds':df_1.loc[(archivo.split('_')[0]),'b'].std(axis=0),
                      'shear_McL':shear_McL,
                      'shear_proyec':shear_proyec,
                      'R_McL':R_McL,
                      'R_Murz':(((shear_McL * a)**2 + (tamano_new * b)**2)**0.5) / (2*tamano_new*math.sin(math.pi/tamano_new)),
                      'area_c':math.pi * R_McL**2,
                      'perim_c':math.pi * R_McL * 2,
                      'twist_McL':a * math.sin(math.radians(alfa)) * math.cos(math.radians(alfa)) / (2 * math.pi * R_McL),
                      'angtw':df_tw['angulo'].mean(axis=0),
                      'angtw_ds':df_tw['angulo'].std(axis=0),
                     }, ignore_index=True)
    
    df_1_todo=pd.concat([df_1_todo, df_1], axis=0)

2dh2A01_SHEETalncris.txt


# Guardar dataframes con nueva información

In [1]:
#path_save=Path('')
#path_save=Path('')
#path_save=Path('')
#path_save=Path('')

#df_2.to_csv(path_save/'dfd_2alncris.txt', header=True, index=False, sep='\t')
#df_1_todo.to_csv(path_save/'dfd_1_todoalncris.txt', header=True, index=True, sep='\t')
#df_angulo_twist.to_csv(path_save/'dfd_angulo_twistalncris.txt', header=True, index=False, sep='\t') 
#df_angulo_conexion.to_csv(path_save/'ac_alncris.txt', header=True, index=False, sep='\t') 


#Carpetas
#prueba2.000001  prueba_aln.000001  prueba_alncris.000001fusion  prueba_stridesolo

In [2]:
#df_angulo_conexion.iloc[:,10:25]
#df_angulo_conexion.iloc[:,25:38]
#print(list(CA['num_res']))